# one-child sample


In [ ]:
cd ""
capture log close
log using "", text replace
clear all
macro drop _all

In [ ]:
use onlychild.dta, clear
eststo clear

In [ ]:
label var childage "Child age"
label var custody_m "Mother custody"
label var p_female "Female plaintiff"
label var caregiver2 "Primary caregivers"
label var malehome2 "Father owns home"
label var female_nojob "Unemployed mother"
label var violence "Claim of domestic violence"
label var ill_m "Mother with serious illness"
label var ill_f "Father with serious illness"
label var addchild_m "Mother has additional children"
label var addchild_f "Father has additional children"
label var rural "Rural areas"
label var consent "Defendant's consent to divorce"

label define child_sex 0 "Boy" 1 "Girl"
label values child_girl child_sex

label define care 0 "Father" 1 "Maternal grandparents" 2 "Paternal grandparents" 3 "Mother + Maternal grandparents" 4 "Mother + Paternal grandparents" 5 "Mother"
label values caregiver2 care

label define home 0 "No" 1 "Yes"
label values malehome2 home

In [ ]:
// for children age 2 and above
quietly logit custody_m i.caregiver2 i.child_girl c.childage c.childage#c.childage i.consent i.p_female i.malehome2 i.female_nojob i.violence i.ill_m i.ill_f i.addchild_m i.addchild_f i.province i.judgeyear if rural==0 & childage>=2, nolog vce(robust)
est store age2_lg_urban
quietly margins, dydx(*) post
est store age2_one_ame_urban

quietly logit custody_m i.caregiver2 i.child_girl c.childage c.childage#c.childage i.consent i.p_female i.female_nojob i.violence i.ill_m i.ill_f i.addchild_m i.addchild_f i.province i.judgeyear if rural==1 & childage>=2, nolog vce(robust)
est store age2_lg_rural
quietly margins, dydx(*) post
est store age2_one_ame_rural

// add customization
///r2
quietly est restore age2_lg_urban
estadd scalar lg_r2 `=e(r2_p)': age2_one_ame_urban
quietly est restore age2_lg_rural
estadd scalar lg_r2 `=e(r2_p)': age2_one_ame_rural
///FE
estadd local fe "Yes": age2_one_ame_urban
estadd local fe "Yes": age2_one_ame_rural

In [ ]:
// table for average marginal effects
esttab age2_one_ame_urban age2_one_ame_rural using stata_ame_onechild_age2.tex, b(2) se(2) label not nobase drop(*.province *.judgeyear) noconstant ///
stats(N fe lg_r2, fmt(0 2) labels("N" "Province and Year FE" "\$R^{2}$")) ///
mtitles("Urban (N = 67,087)" "Rural (N = 89,183)") booktabs nonote ///
addnotes("Robust standard errors in parentheses. * \$P < .05$, ** \$P < .01$, *** \$P <  .001$ (two-tailed tests).") ///
refcat(1.caregiver2 "Primary caregivers (Ref: Father)", nolabel) replace

In [ ]:
// probabilities, urban/rural, by child sex and age
quietly estimates restore age2_lg_urban
quietly margins child_girl, at(childage=(2(1)18)) post
est store m_childsex_age_urban

quietly estimates restore age2_lg_rural
quietly margins child_girl, at(childage=(2(1)18)) post
est store m_childsex_age_rural

esttab m_childsex_age_* using stata_probability_childsex_age2.csv, b(3) not label ci wide nostar collabels("probability" "ci") replace

In [ ]:
// probabilities, urban/rural, by caregiver and homeownership
quietly estimates restore age2_lg_urban
quietly margins caregiver2, at(malehome2=(0 1)) post
est store m_caregiver_home_urban

quietly estimates restore age2_lg_rural
quietly margins caregiver2, post
est store m_caregiver_home_rural

esttab m_caregiver_home_* using stata_probability_caregiver_home_sf.csv, b(3) not label ci wide nostar collabels("probability" "ci") replace

# Multi-child sample


In [ ]:
cd ""
capture log close
log using "", text replace
clear all
macro drop _all

In [ ]:
use multichild.dta, clear
eststo clear

In [ ]:
label var average_age "Average age of siblings"
label var age_gap "Sibling age gap"
label var p_female "Female plaintiff"
label var caregiver2 "Primary caregivers"
label var malehome2 "Father owns home"
label var female_nojob "Unemployed mother"
label var violence "Claim of domestic violence"
label var ill_m "Mother with serious illness"
label var ill_f "Father with serious illness"
label var addchild_m "Mother has additional children"
label var addchild_f "Father has additional children"
label var rural "Rural areas"
label var consent "Defendant's consent to divorce"

label define care 0 "Father" 1 "Maternal grandparents" 2 "Paternal grandparents" 3 "Mother + Maternal grandparents" 4 "Mother + Paternal grandparents" 5 "Mother"
label values caregiver2 care

label define home 0 "No" 1 "Yes"
label values malehome2 home

label define sample_label 0 "Girls and Boys" 1 "Girls" 2 "Boys"
label values sib_com sample_label

label define custody 0 "Father Custody" 1 "Split Custody" 2 "Mother Custody"
label values custody_outcome custody

In [ ]:
// base models, cases involving children under age 2 removed
quietly mlogit custody_outcome i.sib_com i.caregiver2 i.malehome2 c.average_age c.age_gap i.consent i.p_female i.female_nojob i.violence i.ill_m i.ill_f i.addchild_m i.addchild_f i.province i.judgeyear if rural==0, base(0) nolog vce(robust)
est store mlg_urban

quietly mlogit custody_outcome i.sib_com i.caregiver2 c.average_age c.age_gap i.consent i.p_female i.female_nojob i.violence i.ill_m i.ill_f i.addchild_m i.addchild_f i.province i.judgeyear if rural==1, base(0) nolog vce(robust)
est store mlg_rural

In [ ]:
// average marginal effects
quietly estimates restore mlg_urban
quietly margins, dydx(*) predict(outcome(2)) vsquish post
est store two_ame_urban2

quietly estimates restore mlg_urban
quietly margins, dydx(*) predict(outcome(0)) vsquish post
est store two_ame_urban0

quietly estimates restore mlg_urban
quietly margins, dydx(*) predict(outcome(1)) vsquish post
est store two_ame_urban1

quietly estimates restore mlg_rural
quietly margins, dydx(*) predict(outcome(2)) vsquish post
est store two_ame_rural2

quietly estimates restore mlg_rural
quietly margins, dydx(*) predict(outcome(0)) vsquish post
est store two_ame_rural0

quietly estimates restore mlg_rural
quietly margins, dydx(*) predict(outcome(1)) vsquish post
est store two_ame_rural1

In [ ]:
// add customization
///r2
quietly est restore mlg_urban
estadd scalar mlg_r2 `=e(r2_p)': two_ame_urban2
estadd scalar mlg_r2 `=e(r2_p)': two_ame_urban0
estadd scalar mlg_r2 `=e(r2_p)': two_ame_urban1

quietly est restore mlg_rural
estadd scalar mlg_r2 `=e(r2_p)': two_ame_rural2
estadd scalar mlg_r2 `=e(r2_p)': two_ame_rural0
estadd scalar mlg_r2 `=e(r2_p)': two_ame_rural1

///FE
estadd local fe "Yes": two_ame_urban2
estadd local fe "Yes": two_ame_urban0
estadd local fe "Yes": two_ame_urban1

estadd local fe "Yes": two_ame_rural2
estadd local fe "Yes": two_ame_rural0
estadd local fe "Yes": two_ame_rural1

In [ ]:
// table for average marginal effects
esttab two_ame_urban2 two_ame_urban0 two_ame_urban1 two_ame_rural2 two_ame_rural0 two_ame_rural1 using stata_ame_multichild.tex, b(2) se(2) label not nobase drop(*.province *.judgeyear) noconstant ///
stats(N fe mlg_r2, fmt(0 2) labels("N" "Province and Year FE" "\$R^{2}$")) ///
mtitles("Mother" "Father" "Split" "Mother" "Father" "Split") booktabs nonote ///
addnotes("Robust standard errors in parentheses. * \$P < .05$, ** \$P < .01$, *** \$P <  .001$ (two-tailed tests).") ///
refcat(1.caregiver2 "Primary caregivers (Ref: Father)" 1.sib_com "Sibling structure (Ref: Girl and Boy)", nolabel) replace

In [ ]:
// probabilities by sibling structure and urban/rural
quietly estimates restore mlg_urban
quietly margins sib_com, predict(outcome(2)) vsquish post
est store custody_sibling_u_mother

quietly estimates restore mlg_urban
quietly margins sib_com, predict(outcome(0)) vsquish post
est store custody_sibling_u_father

quietly estimates restore mlg_urban
quietly margins sib_com, predict(outcome(1)) vsquish post
est store custody_sibling_u_split

quietly estimates restore mlg_rural
quietly margins sib_com, predict(outcome(2)) vsquish post
est store custody_sibling_r_mother

quietly estimates restore mlg_rural
quietly margins sib_com, predict(outcome(0)) vsquish post
est store custody_sibling_r_father

quietly estimates restore mlg_rural
quietly margins sib_com, predict(outcome(1)) vsquish post
est store custody_sibling_r_split

esttab custody_sibling_u_* custody_sibling_r_* using stata_probability_custody_sibling_m.csv, b(2) not label nostar replace

In [ ]:
// probabilities by caregiver and homeownership
quietly estimates restore mlg_urban
quietly margins caregiver2, at(malehome2=(0 1)) predict(outcome(2)) vsquish post
est store custody_home_u_mother

quietly estimates restore mlg_rural
quietly margins caregiver2, predict(outcome(2)) vsquish post
est store custody_home_r_mother

quietly estimates restore mlg_urban
quietly margins caregiver2, at(malehome2=(0 1)) predict(outcome(0)) vsquish post
est store custody_home_u_father

quietly estimates restore mlg_rural
quietly margins caregiver2, predict(outcome(0)) vsquish post
est store custody_home_r_father

quietly estimates restore mlg_urban
quietly margins caregiver2, at(malehome2=(0 1)) predict(outcome(1)) vsquish post
est store custody_home_u_split

quietly estimates restore mlg_rural
quietly margins caregiver2, predict(outcome(1)) vsquish post
est store custody_home_r_split

esttab custody_home_u_* custody_home_r_* using stata_probability_custody_home_m.csv, b(3) not label ci wide nostar replace